In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
from sklearn import datasets
mnist = datasets.fetch_mldata('MNIST original', data_home='.')

In [3]:
N_train = 20000
N_validation = 4000

X = mnist.data
y = mnist.target
Y = np.eye(10)[y.astype(int)]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=N_train)
X_train, X_validation, Y_train, Y_calidation = train_test_split(X_train, Y_train, test_size=N_validation)

In [4]:
class DNN(object):
    def __init__(self, n_in, n_hiddens, n_out):
        self.n_in = n_in
        self.n_hiddens = n_hiddens
        self.n_out = n_out
        self.weights = []
        self.biases = []

        self._x = None
        self._t = None,
        self._keep_prob = None
        self._sess = None
        self._history = {
            'accuracy': [],
            'loss': []
        }

    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(self, shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)

    def inference(self, x, keep_prob):
        # 入力層 - 隠れ層、隠れ層 - 隠れ層
        for i, n_hidden in enumerate(self.n_hiddens):
            if i == 0:
                input = x
                input_dim = self.n_in
            else:
                input = output
                input_dim = self.n_hiddens[i-1]

            self.weights.append(self.weight_variable([input_dim, n_hidden]))
            self.biases.append(self.bias_variable([n_hidden]))

            h = tf.nn.relu(tf.matmul(
                input, self.weights[-1]) + self.biases[-1])
            output = tf.nn.dropout(h, keep_prob)

        # 隠れ層 - 出力層
        self.weights.append(
            self.weight_variable([self.n_hiddens[-1], self.n_out]))
        self.biases.append(self.bias_variable([self.n_out]))

        y = tf.nn.softmax(tf.matmul(
            output, self.weights[-1]) + self.biases[-1])
        return y

    def loss(self, y, t):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y), axis=1))
        return cross_entropy

    def training(self, loss):
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train_step = optimizer.minimize(loss)
        return train_step

    def accuracy(self, y, t):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy

    def fit(self, X_train, Y_train,
            epochs=100, batch_size=100, p_keep=0.5,
            verbose=1):
        x = tf.placeholder(tf.float32, shape=[None, self.n_in])
        t = tf.placeholder(tf.float32, shape=[None, self.n_out])
        keep_prob = tf.placeholder(tf.float32)

        self._x = x
        self._t = t
        self._keep_prob = keep_prob

        y = self.inference(x, keep_prob)
        loss = self.loss(y, t)
        train_step = self.training(loss)
        accuracy = self.accuracy(y, t)

        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)

        self._sess = sess

        N_train = len(X_train)
        n_batches = N_train // batch_size

        for epoch in range(epochs):
            X_, Y_ = shuffle(X_train, Y_train)

            for i in range(n_batches):
                start = i * batch_size
                end = start + batch_size

                sess.run(train_step, feed_dict={
                    x: X_[start:end],
                    t: Y_[start:end],
                    keep_prob: p_keep
                })
            loss_ = loss.eval(session=sess, feed_dict={
                x: X_train,
                t: Y_train,
                keep_prob: 1.0
            })
            accuracy_ = accuracy.eval(session=sess, feed_dict={
                x: X_train,
                t: Y_train,
                keep_prob: 1.0
            })
            self._history['loss'].append(loss_)
            self._history['accuracy'].append(accuracy_)

            if verbose:
                print('epoch:', epoch,
                      ' loss:', loss_,
                      ' accuracy:', accuracy_)

        return self._history

    def evaluate(self, X_test, Y_test):
        y = self.inference(self._x, self._keep_prob)
        accuracy = self.accuracy(y, self._t)
        return accuracy.eval(session=self._sess, feed_dict={
            self._x: X_test,
            self._t: Y_test,
            self._keep_prob: 1.0
        })


In [5]:
model = DNN(n_in=784,
           n_hiddens=[200, 200, 200],
           n_out=10)
model.fit(X_train, Y_train, epochs=50, batch_size=200, p_keep=0.5)
accuracy = model.evaluate(X_test, Y_test)
print('accuracy: ', accuracy)

epoch: 0  loss: 2.10719  accuracy: 0.248688
epoch: 1  loss: 0.835903  accuracy: 0.7395
epoch: 2  loss: 0.499571  accuracy: 0.852375
epoch: 3  loss: 0.361824  accuracy: 0.89525
epoch: 4  loss: 0.305729  accuracy: 0.910438
epoch: 5  loss: 0.254992  accuracy: 0.926938
epoch: 6  loss: 0.231167  accuracy: 0.932063
epoch: 7  loss: 0.205313  accuracy: 0.93925
epoch: 8  loss: 0.178686  accuracy: 0.946688
epoch: 9  loss: 0.161913  accuracy: 0.952438
epoch: 10  loss: 0.149449  accuracy: 0.955625
epoch: 11  loss: 0.139389  accuracy: 0.958313
epoch: 12  loss: 0.131276  accuracy: 0.960125
epoch: 13  loss: 0.115758  accuracy: 0.965625
epoch: 14  loss: 0.110505  accuracy: 0.967188
epoch: 15  loss: nan  accuracy: 0.100188
epoch: 16  loss: nan  accuracy: 0.100188
epoch: 17  loss: nan  accuracy: 0.100188
epoch: 18  loss: nan  accuracy: 0.100188
epoch: 19  loss: nan  accuracy: 0.100188
epoch: 20  loss: nan  accuracy: 0.100188
epoch: 21  loss: nan  accuracy: 0.100188
epoch: 22  loss: nan  accuracy: 0.1001

FailedPreconditionError: Attempting to use uninitialized value Variable_15
	 [[Node: Variable_15/read = Identity[T=DT_FLOAT, _class=["loc:@Variable_15"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable_15)]]

Caused by op 'Variable_15/read', defined at:
  File "/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/shimazu/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.

In [ ]:
n_in = len(X[0])  # 784
n_hidden = 200
n_out = len(Y[0])  # 10

x = tf.placeholder(tf.float32, shape=[None, n_in])
t = tf.placeholder(tf.float32, shape=[None, n_out])
keep_prob = tf.placeholder(tf.float32)

# 入力層 - 隠れ層
W0 = tf.Variable(tf.truncated_normal([n_in, n_hidden], stddev=0.01))
b0 = tf.Variable(tf.zeros([n_hidden]))
h0 = tf.nn.relu(tf.matmul(x, W0) + b0)
h0_drop = tf.nn.dropout(h0, keep_prob)

# 隠れ層 - 隠れ層
W1 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b1 = tf.Variable(tf.zeros([n_hidden]))
h1 = tf.nn.relu(tf.matmul(h0_drop, W1) + b1)
h1_drop = tf.nn.dropout(h1, keep_prob)

# 隠れ層 - 隠れ層
W2 = tf.Variable(tf.truncated_normal([n_hidden, n_hidden], stddev=0.01))
b2 = tf.Variable(tf.zeros([n_hidden]))
h2 = tf.nn.relu(tf.matmul(h1_drop, W1) + b1)
h2_drop = tf.nn.dropout(h2, keep_prob)

# 入力層 - 隠れ層
W3 = tf.Variable(tf.truncated_normal([n_hidden, n_out], stddev=0.01))
b3 = tf.Variable(tf.zeros([n_out]))
y = tf.nn.softmax(tf.matmul(h2_drop, W3) + b3)

In [ ]:
cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

In [ ]:
epochs = 500
batch_size = 100
n_bath = N // batch_size

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
    X_, Y_ = shuffle(X_train, Y_train)
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        
        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end],
            keep_prob: 0.5
        })

In [ ]:
classified = correct_prediction.eval(session=sess, feed_dict={
    x: X_test,
    t: Y_test
})
prob = y.eval(session=sess, feed_dict={
    x: X_test
})
print('classified: \n', classified)
print()
print('output prob: \n', prob)